In [1]:
# Library/module imports
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
from time import sleep
import pandas as pd

In [2]:
## Pretend to be a browser (https://stackoverflow.com/questions/43590153/http-error-403-forbidden-when-reading-html/43590290#43590290):
header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
}

In [3]:
read_pg = requests.get("https://acharts.co/canada_singles_top_100/2022/36", headers=header)

In [4]:
# Learning soup
# https://www.crummy.com/software/BeautifulSoup/bs4/doc/
soup = BeautifulSoup(read_pg.text, "html.parser")

In [68]:
## Dump in file for fun:
html = soup.prettify("utf-8")
with open("soupDump.html", "wb") as file:
    file.write(html)

In [69]:
# All is in here:
type(soup.findAll("table")[0])

bs4.element.Tag

In [70]:
s = soup.findAll("table")[0]

In [6]:
def song_scrape(soup):
    s = soup.findAll("table")[0]
    top100 = []

    for tr in s.find("tbody").findAll("tr"):
        # tr = s.find("tbody").findAll("tr")[2]

        rank = tr.find('span', {"itemprop": "position"}).text
        rankPrev = tr.find('span', {'class':'Sub subStatsPrev'}).text
        rankPrev = rankPrev.replace("\n","").replace(" ","")
        title = tr.find('span', {'itemprop':'name'}).text

        artist = ""
        first = True
        for artists in tr.findAll('span', {'itemprop':'byArtist'}):
            if first:
                artist += artists.find('span', {'itemprop':'name'}).text
                first = False
            else:
                artist += " and " + artists.find('span', {'itemprop':'name'}).text
        
        top100.append((title, artist, rank, rankPrev))
    return top100

In [7]:
foo = song_scrape(soup)

In [8]:
foo

[('Sunroof', 'Nicky Youre and Dazy', '1', '(2)'),
 ('As It Was', 'Harry Styles', '2', '(1)'),
 ('Hold Me Closer', 'Elton John and Britney Spears', '3', '(new)'),
 ('About Damn Time', 'Lizzo', '4', '(3)'),
 ('Numb', 'Marshmello and Khalid', '5', '(5)'),
 ('I Like You (a Happier Song)', 'Post Malone and Doja Cat', '6', '(6)'),
 ('Late Night Talking', 'Harry Styles', '7', '(7)'),
 ("I'm Good (blue)", 'David Guetta and Bebe Rexha', '8', '(new)'),
 ("I Ain't Worried", 'OneRepublic', '9', '(10)'),
 ('Flowers Need Rain', 'Preston Pablo and Banx + Ranx', '10', '(9)'),
 ('Heat Waves', 'Glass Animals', '11', '(8)'),
 ('Super Freaky Girl', 'Nicki Minaj', '12', '(11)'),
 ('The Kind Of Love We Make', 'Luke Combs', '13', '(14)'),
 ('Cold Heart (pnau Remix)', 'Elton John and Dua Lipa', '14', '(15)'),
 ('Bad Habit', 'Steve Lacy', '15', '(13)'),
 ('Jimmy Cooks', 'Drake and 21 Savage', '16', '(16)'),
 ('Break My Soul', 'Beyoncé', '17', '(12)'),
 ('Staying Alive', 'DJ Khaled and Drake and Lil Baby', '18'

## part 2: lyrics!
Using: https://www.azlyrics.com/

Format song to be: primary artist / title (nospaces, brackets, accents)

In [9]:
import unidecode

# Function to remove special characters
# and store it in another variable
def removeSpecialCharacter(s):
    t = ""
    for i in s:
        if(i.isalpha()):
            t+=i

    return t

def strip_string(s):
    # force lower:
    s = s.lower()
    # unaccent string:
    s = unidecode.unidecode(s)
    # remove anything that isn't a letter and return
    return removeSpecialCharacter(s)

In [21]:
first_artist = foo[2][1].split('and')[0]
first_artist = strip_string(first_artist)
song = foo[2][0]
song = strip_string(song)

lyric_url = "https://www.azlyrics.com/lyrics/" + first_artist + "/" + song + ".html"
lyric_url

'https://www.azlyrics.com/lyrics/eltonjohn/holdmecloser.html'

In [22]:
lyric_url

'https://www.azlyrics.com/lyrics/eltonjohn/holdmecloser.html'

In [23]:
# read page:
read_pg = requests.get(lyric_url, headers=header)

In [24]:
soup = BeautifulSoup(read_pg.text, "html.parser")

In [25]:
## Dump in file for fun:
html = soup.prettify("utf-8")
with open("soupDump2.html", "wb") as file:
    file.write(html)

Appendix

In [24]:
tr = s.find("tbody").findAll("tr")[2]

In [72]:
#Song position!
tr.find('span', {"itemprop": "position"}).text

'3'

In [73]:
# Previous
tr.find('span', {'class':'Sub subStatsPrev'}).text

'\n\n                        (new)\n                    '

In [74]:
# Title
tr.find('span', {'itemprop':'name'}).text

'Hold Me Closer'

In [75]:
# artist(s)
artist = ""
first = True
for artists in tr.findAll('span', {'itemprop':'byArtist'}):
    if first:
        artist += artists.find('span', {'itemprop':'name'}).text
        first = False
    else:
        artist += " and " + artists.find('span', {'itemprop':'name'}).text
artist

'Elton John and Britney Spears'